In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import easyquotation
import json
import requests

In [2]:
allStockDf = pd.read_csv('./all_cleaned.csv')
stockListDf = pd.read_csv('./hk_stock_list.csv')
allStockDf = allStockDf.loc[:, ~allStockDf.columns.str.contains('^Unnamed')]
stockListDf = stockListDf.loc[:, ~stockListDf.columns.str.contains('^Unnamed')]
stockCodeList = stockListDf['code'].tolist()
stockNameList = stockListDf['name'].tolist()
display(allStockDf)
display(stockListDf)

,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate,year,month
0,1,長江和記實業s,2015-12-01,118570.0,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1.0,2015.0,12.0
1,1,長江和記實業s,2016-12-01,33008.0,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1.0,2016.0,12.0
2,1,長江和記實業s,2017-12-01,35100.0,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1.0,2017.0,12.0
3,1,長江和記實業s,2018-12-01,39000.0,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1.0,2018.0,12.0
4,1,長江和記實業s,2019-12-01,39830.0,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1.0,2019.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,83199,南方政銀國債－Ｒs,2017-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2017.0,12.0
12624,83199,南方政銀國債－Ｒs,2018-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2018.0,12.0
12625,87001,匯賢產業信託s,2015-12-01,168.0,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1.0,2015.0,12.0
12626,87001,匯賢產業信託s,2018-12-01,574.0,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1.0,2018.0,12.0


,code,name,price
0,1,長江和記實業s,57.00
1,2,中電控股s,81.50
2,3,香港中華煤氣s,13.76
3,4,九龍倉集團s,13.98
4,5,匯豐控股s,38.80
...,...,...,...
2708,83186,中金金瑞中網－Ｒs,59.75
2709,83188,華夏滬深三百－Ｒs,39.20
2710,83197,華夏Ａ國際通－Ｒs,0.00
2711,83199,南方政銀國債－Ｒs,106.60


In [3]:
#類型變更
allStockDf[allStockDf.columns.difference(['code', 'name' , 'currency' , 'time' ])] = allStockDf[allStockDf.columns.difference(['code', 'name' , 'currency' , 'time' ])].astype(float)
#Convert Quarter end to datetime
allStockDf['time'] = pd.to_datetime(allStockDf['time'])
#Generate a Year and Month column from Quarter end
allStockDf['year'] = allStockDf['time'].dt.year.astype('Int64')
allStockDf['month'] = allStockDf['time'].dt.month.astype('Int64')
allStockDf['code'] = allStockDf['code'].astype(str)
allStockDf['code'] = allStockDf['code'].str.zfill(5)
allStockDf


,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate,year,month
0,00001,長江和記實業s,2015-12-01,118570.0,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1.0,2015,12
1,00001,長江和記實業s,2016-12-01,33008.0,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1.0,2016,12
2,00001,長江和記實業s,2017-12-01,35100.0,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1.0,2017,12
3,00001,長江和記實業s,2018-12-01,39000.0,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1.0,2018,12
4,00001,長江和記實業s,2019-12-01,39830.0,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1.0,2019,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,83199,南方政銀國債－Ｒs,2017-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2017,12
12624,83199,南方政銀國債－Ｒs,2018-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2018,12
12625,87001,匯賢產業信託s,2015-12-01,168.0,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1.0,2015,12
12626,87001,匯賢產業信託s,2018-12-01,574.0,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1.0,2018,12


In [10]:
stockListDf['code'] = stockListDf['code'].astype('str')
stockListDf['code'] = stockListDf['code'].str.zfill(5)
stockListDf

,code,name,price
0,00001,長江和記實業s,57.00
1,00002,中電控股s,81.50
2,00003,香港中華煤氣s,13.76
3,00004,九龍倉集團s,13.98
4,00005,匯豐控股s,38.80
...,...,...,...
2708,83186,中金金瑞中網－Ｒs,59.75
2709,83188,華夏滬深三百－Ｒs,39.20
2710,83197,華夏Ａ國際通－Ｒs,0.00
2711,83199,南方政銀國債－Ｒs,106.60


In [5]:
allStockDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12628 entries, 0 to 12627
Data columns (total 16 columns):
code                12628 non-null object
name                12628 non-null object
time                12401 non-null datetime64[ns]
netProfit           12401 non-null float64
netProfit Growth    11963 non-null float64
eps                 12401 non-null float64
epsGrowth           11777 non-null float64
interestRates       12401 non-null float64
PE                  11958 non-null float64
dividendYield       10765 non-null float64
payoutRatio         11958 non-null float64
netAssetValue       11700 non-null float64
currency            12401 non-null object
rate                12401 non-null float64
year                12401 non-null Int64
month               12401 non-null Int64
dtypes: Int64(2), datetime64[ns](1), float64(10), object(3)
memory usage: 1.6+ MB


In [17]:
stockListDf

,code,name,price
0,00001,長江和記實業s,57.00
1,00002,中電控股s,81.50
2,00003,香港中華煤氣s,13.76
3,00004,九龍倉集團s,13.98
4,00005,匯豐控股s,38.80
...,...,...,...
2708,83186,中金金瑞中網－Ｒs,59.75
2709,83188,華夏滬深三百－Ｒs,39.20
2710,83197,華夏Ａ國際通－Ｒs,0.00
2711,83199,南方政銀國債－Ｒs,106.60


In [18]:
stockFilterDf = stockListDf[stockListDf['price'] >= 1 ]
stockFilterDf

,code,name,price
0,00001,長江和記實業s,57.00
1,00002,中電控股s,81.50
2,00003,香港中華煤氣s,13.76
3,00004,九龍倉集團s,13.98
4,00005,匯豐控股s,38.80
...,...,...,...
2706,83169,領航中國－Ｒs,8.11
2708,83186,中金金瑞中網－Ｒs,59.75
2709,83188,華夏滬深三百－Ｒs,39.20
2711,83199,南方政銀國債－Ｒs,106.60


In [33]:
allStockDf.loc[0]

code                              00001
name                            長江和記實業s
time                2015-12-01 00:00:00
netProfit                        118570
netProfit Growth                 120.11
eps                               36.91
epsGrowth                         58.68
interestRates                      2.55
PE                                 2.61
dividendYield                      2.64
payoutRatio                        6.91
netAssetValue                   101.935
currency                             港元
rate                                  1
year                               2015
month                                12
Name: 0, dtype: object

In [125]:

filterYear = [
    pd.Timestamp(2017,12,1),
    pd.Timestamp(2018,12,1),
    pd.Timestamp(2019,12,1)
]
filterStockFinanceDf = allStockDf[(allStockDf['eps'] > 0) 
           & (allStockDf['epsGrowth'] > 45) 
           & ( allStockDf['code'].isin(stockFilterDf['code'].tolist()) )
           & ( allStockDf['time'].isin(filterYear) )
          ]
strongStock =  stockListDf[
    stockListDf['code'].isin(filterStockFinanceDf.groupby('code')['code'].agg({'count': 'count'}).query('count > 1').index.tolist())]

pd.set_option('display.max_rows', strongStock.shape[0]+1)
display(strongStock)
# display(filterStockFinanceDf.groupby('code').size()>1)
# filterStockFinanceDf[filterStockFinanceDf.groupby('code')['code'].size() > 1]
# display(filterStockFinanceDf.groupby('code')['code'].size() > 1)
# display(filterStockFinanceDf[filterStockFinanceDf.groupby('code')['code'].size() > 1])
# filterdList = list(set(filterStockFinanceDf['code'].tolist()))

# allStockDf[allStockDf['time'] == pd.Timestamp(2018,12,1)]

/Users/kim/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  if sys.path[0] == '':


,code,name,price
12,00014,希慎興業s,25.90
30,00032,港通控股s,11.12
80,00086,新鴻基公司s,3.12
160,00171,銀建國際控股s,1.06
492,00564,鄭州煤礦機械,2.88
511,00586,海螺創業s,36.10
520,00596,浪潮國際s,2.52
550,00631,三一國際s,4.11
577,00665,海通國際s,1.79
602,00691,山水水泥s,2.28


In [121]:
filterStockFinanceDf[filterStockFinanceDf['code']=='00777']

,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate,year,month
3284,00777,網龍s,2018-12-01,621.0,88.91,1.1662,89.14,0.25,17.58,1.22,21.44,10.5743,港元,1.1386,2018,12
3285,00777,網龍s,2019-12-01,902.0,45.21,1.7062,46.31,0.40,12.37,1.90,23.44,10.5743,港元,1.1175,2019,12
